# Uses interaction to push triples to the brain and query it

This notebook assumes you already understand the following notebooks:

* lets-chat.ipynb
* roboGrasp-api.ipynb

In this notebook, we will combine the interaction modeled through EMISSOR with the interaction throught *capsules* with the BRAIN. As auxiliary modules, we will use the *cltl-knowledgeextraction* and *cltl-language-generation*. The *cltl-knowledgeextraction* will extract triples from the utterances either for posting or for querying. In the former case, it also extracts the source perspective from the text. The triples and source perspectives are represented in enriched capsules. We will also use a replier that is include in the *cltl-languagegeneration* package. This replier transfers the response from the BRAIN into natural language expressions and possibly gestures. 

In order to connect to the EMISSOR scenario, we need to align the meta properties of the scenario with the meta data in the capsules. However,  the *cltl-knowledgeextraction* uses a similar data object to keep track of the conversation history. This is a Chat object that needs to be created. Through this Chat object, we keep track of what was said before and deal with coreference to earlier utterances.

Combining EMISSOR and the BRAIN, we can model the interaction in a infite while loop where we go through the following steps:

* We create an EMISSOR scenario at the start of the interaction and a corresponding Chat object to keep track of the dialogue history
* while the user does not stop:
    * create TextSignals for each utterance from the user and store these in an EMISSOR scenario
    * add the utterance also to the Chat instance
    * process the latest utterance in the Chat object using the *cltl-knowledgeextraction* to get triples and perspectives
    * create a capsule from the triples, perspectives and the text signal meta data
    * post the triple to the BRAIN
    * get the answer or throughts as a response from the BRAIN
    * verbalise the answer or thoughts
    * create a new TextSignal for the system response and add it to the EMISSOR scenario
* When the user stops, we save the scenario in the EMISSOR format.



Before running, start GraphDB and make sure that there is a sandbox repository.
GraphDB can be downloaded from:

https://graphdb.ontotext.com

## Import the necessary modules

In [1]:
import json
import os
import time
import uuid
from datetime import date
from datetime import datetime
from random import getrandbits, choice
import pathlib
import pprint

# general imports for EMISSOR and the BRAIN
import emissor as em
import requests
from cltl import brain
from cltl.brain.long_term_memory import LongTermMemory
from cltl.brain.utils.helper_functions import brain_response_to_json
from cltl.combot.backend.api.discrete import UtteranceType
from cltl.reply_generation.data.sentences import GREETING, ASK_NAME, ELOQUENCE, TALK_TO_ME
from cltl.reply_generation.lenka_replier import LenkaReplier
from cltl.triple_extraction.api import Chat, UtteranceHypothesis
from emissor.persistence import ScenarioStorage
from emissor.representation.annotation import AnnotationType, Token, NER
from emissor.representation.container import Index
from emissor.representation.scenario import Modality, ImageSignal, TextSignal, Mention, Annotation, Scenario

/Users/piek/PycharmProjects/cltl-chatbots/venv/lib/python3.7/site-packages/rdflib_jsonld/__init__.py:12: DeprecationWarning: The rdflib-jsonld package has been integrated into rdflib as of rdflib==6.0.1.  Please remove rdflib-jsonld from your project's dependencies.
  DeprecationWarning,
[nltk_data] Downloading package punkt to /Users/piek/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Import the chatbot utility functions

In [2]:
import sys
import os

src_path = os.path.abspath(os.path.join('..'))
if src_path not in sys.path:
    sys.path.append(src_path)

#### The next utils are needed for the interaction and creating triples and capsules
import chatbots.util.driver_util as d_util
import chatbots.util.capsule_util as c_util
import chatbots.intentions.talk as talk
import chatbots.intentions.get_to_know_you as friend

## Standard initialisation of a scenario

We initialise a scenario in the standard way by creating a unique folder and setting the AGENT and HUMAN_NAME and HUMAN_ID variables. Throughout this scenario, the HUMAN_NAME and HUMAN_ID will be used as the source for the utterances.

In [3]:
from random import getrandbits
import requests
##### Setting the location
place_id = getrandbits(8)
location = None
try:
    location = requests.get("https://ipinfo.io").json()
except:
    print("failed to get the IP location")
    
##### Setting the agents
AGENT = "Leolani2"
HUMAN_NAME = "Stranger"
HUMAN_ID = "stranger"

### The name of your scenario
scenario_id = datetime.today().strftime("%Y-%m-%d-%H:%M:%S")

### Specify the path to an existing data folder where your scenario is created and saved as a subfolder
# Find the repository root dir
parent, dir_name = (d_util.__file__, "_")
while dir_name and dir_name != "src":
    parent, dir_name = os.path.split(parent)
root_dir = parent
scenario_path = os.path.abspath(os.path.join(root_dir, 'data'))

if not os.path.exists(scenario_path) :
    os.mkdir(scenario_path)
    print("Created a data folder for storing the scenarios", scenario_path)

### Define the folders where the images and rdf triples are saved
imagefolder = scenario_path + "/" + scenario_id + "/" + "image"
rdffolder = scenario_path + "/" + scenario_id + "/" + "rdf"

### Create the scenario folder, the json files and a scenarioStorage and scenario in memory
scenarioStorage = d_util.create_scenario(scenario_path, scenario_id)
scenario_ctrl = scenarioStorage.create_scenario(scenario_id, int(time.time() * 1e3), None, AGENT)

Directories for 2021-11-19-09:49:18 created in /Users/piek/PycharmProjects/cltl-chatbots/data


## Specifying the BRAIN

We specify the BRAIN in GraphDB and use the scenario path just defined for storing the RDF triple produced in EMISSOR.

If you set *clear_all* to *True*, the sandbox triple store is emptied (memory erased) and the basic ontological models are reloaded. Setting it to *False* means you add things to the current memory.

In [4]:
log_path = pathlib.Path(rdffolder)
my_brain = brain.LongTermMemory(address="http://localhost:7200/repositories/sandbox",
                                log_dir=log_path,
                                clear_all=True)


2021-11-19 09:49:28,088 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Booted
2021-11-19 09:49:31,186 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Uploading ontology to brain
2021-11-19 09:49:33,761 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Booted
2021-11-19 09:49:33,763 -     INFO -  cltl.brain.basic_brain.LocationReasoner - Booted
2021-11-19 09:49:33,766 -     INFO -      cltl.brain.basic_brain.TypeReasoner - Booted
2021-11-19 09:49:33,769 -     INFO -   cltl.brain.basic_brain.TrustCalculator - Booted
2021-11-19 09:49:34,009 -     INFO -   cltl.brain.basic_brain.TrustCalculator - Computed trust for all known agents


## Create an instance of a replier

In [5]:
replier = LenkaReplier()

2021-11-19 09:49:38,797 -     INFO -   cltl.reply_generation.api.LenkaReplier - Booted


## Establish the speaker identity

In [6]:
#### Small sequence to learn name of speaker
initial_prompt = f"{choice(GREETING)} {choice(ASK_NAME)} {HUMAN_NAME}?"
print(AGENT + ": " + initial_prompt)
textSignal = d_util.create_text_signal(scenario_ctrl, initial_prompt)
scenario_ctrl.append_signal(textSignal)

#### Get name from person 
HUMAN_NAME, HUMAN_ID = friend.get_a_name_and_id(scenario_ctrl, AGENT)
HUMAN_ID = HUMAN_NAME  ### Hack because we cannot force the namespace through capsules, name and identity are the same till this is fixed

print("Name:", HUMAN_NAME)
print("Id:", HUMAN_ID)
                
capsule = c_util.scenario_utterance_to_capsule(scenario_ctrl,place_id,location, textSignal,HUMAN_ID,AGENT,"know", HUMAN_ID)

name_thoughts = my_brain.update(capsule, reason_types=True, create_label=True)

#pprint.pprint(capsule)

Leolani2: Hi! I've told you my name, but what about yours? Stranger?



 Piek


Piek
Leolani2: So your name is Piek?



 yes


Name: Piek
Id: Piek
2021-11-19 09:50:21,526 -  WARNING -      cltl.brain.basic_brain.TypeReasoner - Failed to query Wikidata: HTTPSConnectionPool(host='query.wikidata.org', port=443): Read timed out. (read timeout=3)
2021-11-19 09:50:22,463 -     INFO -      cltl.brain.basic_brain.TypeReasoner - Reasoned type of Piek to: None
2021-11-19 09:50:22,514 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Triple in statement: leolani2_know_piek [person_->_])
2021-11-19 09:50:22,562 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Entity Novelty: new subject - new object 
2021-11-19 09:50:24,909 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Negation Conflicts: piek on November,2021 about UNDERSPECIFIED
2021-11-19 09:50:24,961 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Gaps: 26 gaps as subject: e.g. travel-to location - 15 gaps as object: e.g. be-parent-of person
2021-11-19 09:50:25,012 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Gaps: 25 gaps as 

## Initialise a chat with the HUMAN_ID to keep track of the dialogue history

In [7]:
chat = Chat(HUMAN_ID)

2021-11-19 09:51:28,686 -     INFO - cltl.triple_extraction.api.Chat (Piek)              000 - << Start of Chat with Piek >>


## Start the interaction

In [8]:
print_details=True


#### Initial prompt by the system from which we create a TextSignal and store it
initial_prompt = f"{choice(TALK_TO_ME)}"
print(AGENT + ": " + initial_prompt)
textSignal = d_util.create_text_signal(scenario_ctrl, initial_prompt)
scenario_ctrl.append_signal(textSignal)

utterance = ""
#### Get input and loop
while not (utterance.lower() == 'stop' or utterance.lower() == 'bye'):
    ###### Getting the next input signals
    utterance = input('\n')
    print(HUMAN_NAME + ": " + utterance)
    textSignal = d_util.create_text_signal(scenario_ctrl, utterance)
    scenario_ctrl.append_signal(textSignal)

    #### Process input and generate reply
    
    capsule, reply = talk.process_text_and_reply(scenario_ctrl,
                           place_id,
                           location,
                           HUMAN_ID,
                           textSignal,
                           chat,
                           replier,
                           my_brain,
                           print_details)

    print(AGENT + ": " + reply)
    textSignal = d_util.create_text_signal(scenario_ctrl, reply)
    scenario_ctrl.append_signal(textSignal)


Leolani2: Do you have any gossip?



 I like cats


Piek: I like cats
2021-11-19 09:52:41,880 -     INFO -               cltl.triple_extraction.api - Started POS tagger
2021-11-19 09:52:41,881 -     INFO -               cltl.triple_extraction.api - Started NER tagger
2021-11-19 09:52:41,888 -     INFO -               cltl.triple_extraction.api - Loaded grammar
2021-11-19 09:52:43,205 -     INFO - cltl.triple_extraction.api.Chat (Piek)              001 -       Piek: "I like cats"
Triple:
{'object': {'label': 'cats', 'type': ['noun.animal']},
 'predicate': {'label': 'like', 'type': ['verb.emotion']},
 'subject': {'label': 'Piek', 'type': ['agent']}}
Perspective:
{'certainty': 1,
 'emotion': <Emotion.NEUTRAL: 7>,
 'polarity': 1,
 'sentiment': '0.75'}
Capsule:
{'author': 'Piek',
 'chat': '2021-11-19-09:49:18',
 'city': 'Amsterdam',
 'context_id': 'Leolani2',
 'country': 'NL',
 'date': datetime.date(2021, 11, 19),
 'object': {'label': 'cats', 'type': 'noun.animal'},
 'objects': [],
 'people': [],
 'perspective': {'certainty': 1,
            


 What do I like?


Piek: What do I like?
2021-11-19 09:53:29,051 -     INFO - cltl.triple_extraction.api.Chat (Piek)              002 -       Piek: "What do I like?"
Triple:
{'object': {'label': '', 'type': []},
 'predicate': {'label': 'like', 'type': ['verb.emotion']},
 'subject': {'label': 'Piek', 'type': ['agent']}}
Perspective:
None
Capsule:
{'author': 'Piek',
 'chat': '2021-11-19-09:49:18',
 'city': 'Amsterdam',
 'context_id': 'Leolani2',
 'country': 'NL',
 'date': datetime.date(2021, 11, 19),
 'object': {'label': '', 'type': []},
 'objects': [],
 'people': [],
 'place': 'Amsterdam',
 'place_id': 225,
 'position': '0-15',
 'predicate': {'label': 'like', 'type': 'verb.emotion'},
 'region': 'North Holland',
 'subject': {'label': 'Piek', 'type': 'agent'},
 'turn': 'fa415081-3a4b-43f4-83dd-3db22bd0df49',
 'utterance': 'What do I like?',
 'utterance_type': <UtteranceType.QUESTION: 1>}
2021-11-19 09:53:29,056 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Triple in question: piek_like_? [agent_->

...

Piek: What does Fred like?
2021-11-19 09:54:31,163 -     INFO - cltl.triple_extraction.api.Chat (Piek)              005 -       Piek: "What does Fred like?"
Triple:
{'object': {'label': '', 'type': []},
 'predicate': {'label': 'do', 'type': ['verb.social']},
 'subject': {'label': 'Fred-like', 'type': ['person', 'noun.cognition']}}
Perspective:
None
Capsule:
{'author': 'Piek',
 'chat': '2021-11-19-09:49:18',
 'city': 'Amsterdam',
 'context_id': 'Leolani2',
 'country': 'NL',
 'date': datetime.date(2021, 11, 19),
 'object': {'label': '', 'type': []},
 'objects': [],
 'people': [],
 'place': 'Amsterdam',
 'place_id': 225,
 'position': '0-20',
 'predicate': {'label': 'do', 'type': 'verb.social'},
 'region': 'North Holland',
 'subject': {'label': 'Fred-like', 'type': 'person'},
 'turn': '0209071d-005d-4da7-b3f1-c3724b1dcf6a',
 'utterance': 'What does Fred like?',
 'utterance_type': <UtteranceType.QUESTION: 1>}
2021-11-19 09:54:31,176 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Tr


 stop


Piek: stop
2021-11-19 09:54:47,401 -     INFO - cltl.triple_extraction.api.Chat (Piek)              006 -       Piek: "stop"


Couldn't parse input


Leolani2: And thus, I die


## Save the scenario data

In [9]:
scenario_ctrl.scenario.ruler.end = int(time.time() * 1e3)
scenarioStorage.save_scenario(scenario_ctrl)

## End of notebook